In [1]:
import add_packages
import os
from pprint import pprint
import pandas as pd
from operator import itemgetter

from toolkit import sql
from toolkit.langchain import (
	text_embedding_models, stores, prompts, output_parsers, agents, runnables,
	chains, models, tools, graphs
)

In [2]:
llm = models.chat_openai
vectorstore = stores.faiss.FAISS
embeddings = text_embedding_models.OpenAIEmbeddings()

# [How-to](https://python.langchain.com/v0.2/docs/how_to/)

## chain runnables


## stream runnables


## invoke runnables in parallel


## add default invocation args to runnables


## turn any function into a runnable


## pass through inputs from one chain step to the next


## configure runnable behavior at runtime


## add message history (memory) to a chain


## route between sub-chains


In [65]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda

#*==============================================================================
categories = ['cars', 'animals']

chain_1 = PromptTemplate.from_template(
	"""You are an expert in cars. \
Always answer questions starting with "As a car expert, I can tell you that". \
Respond to the following question:

Question: {question}
Answer:"""
) | llm
chain_2 = PromptTemplate.from_template(
	"""You are an expert in animals. \
Always answer questions starting with "As an animal expert, I can tell you that". \
Respond to the following question:

Question: {question}
Answer:"""
) | llm

rules = {
	"cars": chain_1,
	"animals": chain_2,
}
#*==============================================================================
def create_prompt_tpl_extract_category(categories: list[str]):
	categories = ", ".join([f"{cate}" for cate in categories])
 
	prompt_tpl_extract_category = """\
You will be given a list of categories and a question. Your task is to determine which category the question falls into and respond with only the name of that category. Do not include any other words in your response.

Here is the list of categories:
<categories>
{{CATEGORIES}}
</categories>

Here is the question to classify:
<question>
{{QUESTION}}
</question>

<example>
<categories>
science, history, math, literature
</categories>

<question>
What is the chemical formula for water?
</question>

science
</example>

Now classify the provided question into one of the given categories. Remember, respond with only a single word - the name of the category.

<question>
{question}
</question>

Classification:
"""

	prompt_tpl_extract_category = prompt_tpl_extract_category.replace("{{CATEGORIES}}", categories)
	return prompt_tpl_extract_category

prompt_tpl_extract_category = create_prompt_tpl_extract_category(categories)
prompt_extract_category = PromptTemplate.from_template(prompt_tpl_extract_category)
chain_extract_category = (
	prompt_extract_category
	| llm
	| StrOutputParser()
)

def route_chains(chain_vars):
	for cat, chain in rules.items():
		if cat.lower() == chain_vars["category"]:
			return chain

chain_routed = (
	{
		"question": lambda x: x["question"],
		"category": chain_extract_category,
	}
	| RunnableLambda(route_chains)
)

In [93]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, Runnable
from langchain_core.language_models.chat_models import  BaseChatModel

class MyRoutableChain:
    def __init__(
      self, 
      categories: list[str], 
      chains: dict[str, Runnable],
      llm: BaseChatModel,
    ):
        self.llm = llm
      
        self.categories = categories
        self.chains = chains
        
        self.prompt_tpl_extract_category = self.create_prompt_tpl_extract_category()
        self.prompt_extract_category = PromptTemplate.from_template(self.prompt_tpl_extract_category)
        
        self.chain_extract_category = (
            self.prompt_extract_category
            | self.llm
            | StrOutputParser()
        )
        
        self.chain_routable = (
            {
                "question": lambda x: x["question"],
                "category": self.chain_extract_category,
            }
            | RunnableLambda(self.route_chains)
        )

    def create_prompt_tpl_extract_category(self):
        categories = ", ".join([f"{cate}" for cate in self.categories])
        prompt_tpl_extract_category = """\
        You will be given a list of categories and a question. Your task is to determine which category the question falls into and respond with only the name of that category. Do not include any other words in your response.

        Here is the list of categories:
        <categories>
        {{CATEGORIES}}
        </categories>

        Here is the question to classify:
        <question>
        {{QUESTION}}
        </question>

        <example>
        <categories>
        science, history, math, literature
        </categories>

        <question>
        What is the chemical formula for water?
        </question>

        science
        </example>

        Now classify the provided question into one of the given categories. Remember, respond with only a single word - the name of the category.

        <question>
        {question}
        </question>

        Classification:
        """

        prompt_tpl_extract_category = prompt_tpl_extract_category.replace("{{CATEGORIES}}", categories)
        return prompt_tpl_extract_category

    def route_chains(self, chain_vars):
        for cat, chain in self.chains.items():
            if cat.lower() == chain_vars["category"]:
                return chain
    
    def get_chain(self):
        return self.chain_routable



#*==============================================================================
categories = ['cars', 'animals']

chain_1 = PromptTemplate.from_template(
	"""You are an expert in cars. \
Always answer questions starting with "As a car expert, I can tell you that". \
Respond to the following question:

Question: {question}
Answer:"""
) | llm
chain_2 = PromptTemplate.from_template(
	"""You are an expert in animals. \
Always answer questions starting with "As an animal expert, I can tell you that". \
Respond to the following question:

Question: {question}
Answer:"""
) | llm

rules = {
	"cars": chain_1,
	"animals": chain_2,
}

my_chain_routable = MyRoutableChain(categories, rules, llm).get_chain()

In [94]:
my_chain_routable.invoke({
  "question": "What is the fastest car?"
})

AIMessage(content='As a car expert, I can tell you that the fastest car currently is the Bugatti Chiron Super Sport 300+, which has a top speed of over 300 mph.', response_metadata={'finish_reason': 'stop'}, id='run-104e1284-cc20-4e62-883b-14e15085b26e-0')

## create a dynamic (self-constructing) chain


## inspect runnables


## add fallbacks to a runnable

# Todos

- [ ] _